
### url for challenge : https://www.hackerearth.com/challenges/competitive/garden-nerd-data-science-competition/

In [0]:
# !wget https://he-public-data.s3-ap-southeast-1.amazonaws.com/HE_Challenge_data.zip

In [0]:
# !unzip HE_Challenge_data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


## Data Pipeline

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [3]:
train.tail(3)

,image_id,category
18537,18537,78
18538,18538,74
18539,18539,29


In [4]:
test.head(3)

,image_id,category
0,18540,NaN
1,18541,NaN
2,18542,NaN


In [5]:
sample.head(3)

,image_id,category
0,18540,0
1,18541,0
2,18542,0


In [6]:
train = train.values

In [7]:
folders, num_samples = np.unique(train[:, 1], return_counts=True)

In [8]:
# make subdirectory of each class
for folder in folders:
  p = os.path.join('data/train', str(folder))
  if not os.path.isdir(p):
    os.mkdir(p)

In [9]:
for i in range(train.shape[0]):
  dest = os.path.join('data/train/', str(train[i][1]))
  src = os.path.join('data/train', str(train[i][0]) + '.jpg')
  shutil.move(src, dest)

#### validation - training split

In [10]:
SPLIT = 0.90 # 95% for training 5% for validation

In [11]:
# create the validation directory
if not os.path.isdir('data/val'):
  os.mkdir('data/val')


In [12]:
# creating subdirectories (of each category) in validation directory
for folder in folders:
  p = os.path.join('data/val', str(folder))
  if not os.path.isdir(p):
    os.mkdir(p)


In [13]:
# move the 3% data from training directory to validation directory
for f in folders:
  path = 'data/train/'+str(f)
  imgs = os.listdir(path)
  split_size = int(SPLIT*len(imgs))
  file_to_move = imgs[split_size:]
  
  for img_f in file_to_move:
    src = os.path.join(path, img_f)
    dest = os.path.join('data/val/'+str(f), img_f)
    shutil.move(src, dest)


###Data Augmentation


In [3]:
from keras.preprocessing.image import ImageDataGenerator

In [4]:
manipulated_path = "./data/manipulated/"

In [5]:
# for i in range(103, 151, 1):
#     for s in os.listdir(manipulated_path):
#         new_cat = os.path.join(manipulated_path, s, str(i))
#         if not os.path.isdir(new_cat):
#             os.mkdir(new_cat)

In [6]:
os.path.isdir?

Signature: os.path.isdir(path, /)
Docstring: Return true if the pathname refers to an existing directory.
Type:      builtin_function_or_method


In [7]:
train_img_gen = ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range = 60,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
)
val_img_gen = ImageDataGenerator(
    rescale = 1/255.0,
)

In [8]:
train_gen = train_img_gen.flow_from_directory(
    'data/main/train',
    batch_size = 8,
)
val_gen = val_img_gen.flow_from_directory(
    'data/main/val/',
)

Found 16638 images belonging to 102 classes.
Found 1902 images belonging to 102 classes.


In [9]:
train_gen_m = train_img_gen.flow_from_directory(
    'data/manipulated/train',
    batch_size = 8,
)
val_gen_m = val_img_gen.flow_from_directory(
    'data/manipulated/val',
)

Found 16638 images belonging to 150 classes.
Found 1902 images belonging to 150 classes.


In [10]:
for (x, y) in train_gen:
  print(x.shape, y.shape)
  break

(8, 256, 256, 3) (8, 102)


In [11]:
for (x, y) in train_gen_m:
  print(x.shape, y.shape)
  break

(8, 256, 256, 3) (8, 150)


In [0]:
# train_gen.class_indices

In [0]:
# val_gen.class_indices

# Claasifiers Models


In [12]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *

In [13]:
# adam = Adam(lr=0.0003)

####  Callback and Checkpoints


In [14]:
from keras.callbacks import Callback,  EarlyStopping

In [15]:
# early stopping
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.05, patience=3)

# models checkpoints-- saves best only 
cp_efficient = keras.callbacks.ModelCheckpoint('efficient_net_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_xception = keras.callbacks.ModelCheckpoint('xception_net_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_vgg = keras.callbacks.ModelCheckpoint('vgg.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_resnet = keras.callbacks.ModelCheckpoint('resnet.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)
cp_nas = keras.callbacks.ModelCheckpoint('nas_net.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)

## Models Architecture

### 1. EfficientNet

In [17]:
# ! pip install keras_efficientnets
from keras_efficientnets import EfficientNetB0

In [18]:
# model for 102
efficient = EfficientNetB0((256, 256, 3), classes=102, weights=None)

In [20]:
# model for 150 class
efficient_m = EfficientNetB0((256, 256, 3), classes=150, weights=None)

In [20]:
# efficient.summary()

In [21]:
# # only last ten layers of both net are trainable
# for layer in efficient.layers[:-10]:
#     layer.trainable =False
# for layer in efficient_m.layers[:-10]:
#     layer.trainable =False

In [26]:
efficient.summary()

___________________
conv2d_41 (Conv2D)              (None, 16, 16, 672)  75264       add_5[0][0]                      
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 16, 16, 672)  2688        conv2d_41[0][0]                  
__________________________________________________________________________________________________
swish_31 (Swish)                (None, 16, 16, 672)  0           batch_normalization_31[0][0]     
__________________________________________________________________________________________________
depthwise_conv2d_11 (DepthwiseC (None, 16, 16, 672)  16800       swish_31[0][0]                   
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 16, 16, 672)  2688        depthwise_conv2d_11[0][0]        
_________________________________________________________________________________________

In [27]:
efficient_m.summary()

___________________
conv2d_171 (Conv2D)             (None, 16, 16, 672)  75264       add_23[0][0]                     
__________________________________________________________________________________________________
batch_normalization_129 (BatchN (None, 16, 16, 672)  2688        conv2d_171[0][0]                 
__________________________________________________________________________________________________
swish_129 (Swish)               (None, 16, 16, 672)  0           batch_normalization_129[0][0]    
__________________________________________________________________________________________________
depthwise_conv2d_43 (DepthwiseC (None, 16, 16, 672)  16800       swish_129[0][0]                  
__________________________________________________________________________________________________
batch_normalization_130 (BatchN (None, 16, 16, 672)  2688        depthwise_conv2d_43[0][0]        
_________________________________________________________________________________________

In [28]:
# ### adding layers for flower classification
# eff_avg = GlobalAveragePooling2D()(efficient.output)
# eff_fc_1 = Dense(256, activation='relu')(eff_avg)
# eff_fc_2 = Dense(128, activation='relu')(eff_fc_1)
# eff_d = Dropout(0.25)(eff_fc_2)
# eff_fc_3 = Dense(102, activation='softmax')(eff_d)

# ### adding layers for flower classification
# # eff_avg_m = GlobalAveragePooling2D()(efficient_m.output)
# # eff_fc_1_m = Dense(256, activation='relu')(eff_avg_m)
# # eff_fc_2_m = Dense(128, activation='relu')(eff_fc_1_m)
# # eff_d_m = Dropout(0.25)(eff_fc_2_m)
# # eff_fc_3_m = Dense(150, activation='softmax')(eff_d_m)

In [30]:
model_1 = efficient #Model(efficient.input, outputs  = eff_fc_3)
model_2 = efficient_m #Model(efficient_m.input, outputs  = eff_fc_3_m)

In [31]:
model_1.summary()

___________________
conv2d_41 (Conv2D)              (None, 16, 16, 672)  75264       add_5[0][0]                      
__________________________________________________________________________________________________
batch_normalization_31 (BatchNo (None, 16, 16, 672)  2688        conv2d_41[0][0]                  
__________________________________________________________________________________________________
swish_31 (Swish)                (None, 16, 16, 672)  0           batch_normalization_31[0][0]     
__________________________________________________________________________________________________
depthwise_conv2d_11 (DepthwiseC (None, 16, 16, 672)  16800       swish_31[0][0]                   
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 16, 16, 672)  2688        depthwise_conv2d_11[0][0]        
_________________________________________________________________________________________

In [32]:
model_2.summary()

___________________
conv2d_171 (Conv2D)             (None, 16, 16, 672)  75264       add_23[0][0]                     
__________________________________________________________________________________________________
batch_normalization_129 (BatchN (None, 16, 16, 672)  2688        conv2d_171[0][0]                 
__________________________________________________________________________________________________
swish_129 (Swish)               (None, 16, 16, 672)  0           batch_normalization_129[0][0]    
__________________________________________________________________________________________________
depthwise_conv2d_43 (DepthwiseC (None, 16, 16, 672)  16800       swish_129[0][0]                  
__________________________________________________________________________________________________
batch_normalization_130 (BatchN (None, 16, 16, 672)  2688        depthwise_conv2d_43[0][0]        
_________________________________________________________________________________________

#### compilation and training

In [33]:
# early stopping
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.005, mode='max', patience=10)
# models checkpoints-- saves best only 
cp_efficient = keras.callbacks.ModelCheckpoint('efficient_net.h5', monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period = 1)

cp_efficient_m = keras.callbacks.ModelCheckpoint('efficient_net_m.h5', monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period = 1)

In [34]:
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model_1_hist = model_1.fit_generator(train_gen, steps_per_epoch=len(train_gen), validation_data=val_gen, validation_steps=len(val_gen), epochs=100, callbacks=[es, cp_efficient])


Epoch 1/100
2080/2080 [==============================] - 538s 259ms/step - loss: 1.4213 - accuracy: 0.6542 - val_loss: 0.0705 - val_accuracy: 0.9564


KeyboardInterrupt: 

In [25]:
# def plot_loss_accuracy(hist):
#   # plot loss
#   print("training_loss =", hist.history['loss'][-1],  "   val_loss =", hist.history['val_loss'][-1])
#   print("training_acc =", hist.history['accuracy'][-1]*100, "   val_acc =", hist.history['val_acc'][-1]*100)
#   plt.plot(hist.history['val_loss'], label='val_loss')
#   plt.plot(hist.history['loss'], label='training_loss')

#   # plot accuracy
#   plt.plot(hist.history['val_acc'], label='val_acc')
#   plt.plot(hist.history['accuracy'], label='training_acc')
#   plt.legend()
#   plt.show()

In [26]:
# plot_loss_accuracy(model_1_hist)

In [26]:
model = load_model("efficient_net.h5")

In [27]:
model.evaluate_generator(val_gen, steps=len(val_gen))

[0.006727680563926697, 0.9616193771362305]

In [28]:
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model_2_hist = model_2.fit_generator(train_gen_m, steps_per_epoch=len(train_gen_m), validation_data=val_gen_m, validation_steps=len(val_gen_m), epochs=100, callbacks=[es, cp_efficient_m])

Epoch 1/100
2080/2080 [==============================] - 557s 268ms/step - loss: 2.0812 - accuracy: 0.4864 - val_loss: 1.0755 - val_accuracy: 0.7902
Epoch 2/100
2080/2080 [==============================] - 350s 168ms/step - loss: 0.9160 - accuracy: 0.7482 - val_loss: 0.5934 - val_accuracy: 0.8270
Epoch 3/100
2080/2080 [==============================] - 288s 138ms/step - loss: 0.6838 - accuracy: 0.8107 - val_loss: 0.4699 - val_accuracy: 0.9054
Epoch 4/100
2080/2080 [==============================] - 287s 138ms/step - loss: 0.5713 - accuracy: 0.8410 - val_loss: 0.0739 - val_accuracy: 0.9038
Epoch 5/100
2080/2080 [==============================] - 287s 138ms/step - loss: 0.4918 - accuracy: 0.8677 - val_loss: 0.0206 - val_accuracy: 0.9264
Epoch 6/100
2080/2080 [==============================] - 290s 139ms/step - loss: 0.4346 - accuracy: 0.8814 - val_loss: 0.2412 - val_accuracy: 0.8728
Epoch 7/100
2080/2080 [==============================] - 295s 142ms/step - loss: 0.4153 - accuracy: 0.8884

In [30]:
model2 = load_model("efficient_net_m.h5")

In [31]:
model2.evaluate_generator(val_gen_m, steps=len(val_gen_m))

[0.34759804606437683, 0.9584648013114929]

### 2.VGG19 


In [0]:
vgg = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [0]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
### adding layers for flower classification
vgg_avg = GlobalAveragePooling2D()(vgg.output)
vgg_fc_1 = Dense(256, activation='relu')(vgg_avg)
vgg_fc_2 = Dense(128, activation='relu')(vgg_fc_1)
vgg_d = Dropout(0.25)(vgg_fc_2)
vgg_fc_3 = Dense(102, activation='softmax')(vgg_d)

In [0]:
vgg = Model(vgg.input, outputs  = vgg_fc_3)

In [0]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
vgg.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
vgg_hist = vgg.fit_generator(train_gen, steps_per_epoch=18305//16, validation_data=val_gen, validation_steps=235//16, epochs=20, callbacks=[es, cp_vgg])

W0819 13:57:12.817914 139621115123584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
1144/1144 [==============================] - 772s 675ms/step - loss: 4.1744 - acc: 0.0978 - val_loss: 3.2947 - val_acc: 0.2881
Epoch 2/20
1144/1144 [==============================] - 757s 661ms/step - loss: 2.3881 - acc: 0.4337 - val_loss: 1.4322 - val_acc: 0.6626
Epoch 3/20
1144/1144 [==============================] - 756s 661ms/step - loss: 1.4062 - acc: 0.6495 - val_loss: 1.0895 - val_acc: 0.7241
Epoch 4/20
1144/1144 [==============================] - 756s 661ms/step - loss: 0.9432 - acc: 0.7561 - val_loss: 0.5591 - val_acc: 0.8276
Epoch 5/20
1144/1144 [==============================] - 757s 661ms/step - loss: 0.7069 - acc: 0.8145 - val_loss: 0.5660 - val_acc: 0.8548
Epoch 6/20
1144/1144 [==============================] - 757s 661ms/step - loss: 0.5448 - acc: 0.8559 - val_loss: 0.3157 - val_acc: 0.9138
Epoch 7/20
1144/1144 [==============================] - 756s 661ms/step - loss: 0.4480 - acc: 0.8804 - val_loss: 0.3707 - val_acc: 0.9089
Epoch 8/20
1144/1144 [============

### 3. XceptionNet

In [0]:
from keras.applications.xception import Xception

In [0]:
xception = Xception(include_top = False, weights = 'imagenet', input_shape = (256, 256, 3))

83689472/83683744 [==============================] - 1s 0us/step


In [0]:
xception.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
xc_avg = GlobalAveragePooling2D()(xception.output)
xc_fc_1 = Dense(256, activation='relu')(xc_avg)
xc_fc_2 = Dense(128, activation='relu')(xc_fc_1)
xc_d = Dropout(0.25)(xc_fc_2)
xc_fc_3 = Dense(102, activation='softmax')(xc_d)

In [0]:
xception = Model(xception.input, outputs  = xc_fc_3)

In [0]:
xception.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
xception_hist = xception.fit_generator(train_gen, steps_per_epoch=18305//16, validation_data=val_gen, validation_steps=235//16, epochs=20, callbacks=[es, cp_xception])

Epoch 1/20
1144/1144 [==============================] - 1048s 916ms/step - loss: 1.9454 - acc: 0.5909 - val_loss: 0.4648 - val_acc: 0.8876
Epoch 2/20
1144/1144 [==============================] - 1033s 903ms/step - loss: 0.5133 - acc: 0.8881 - val_loss: 0.1565 - val_acc: 0.9631
Epoch 3/20
1144/1144 [==============================] - 1038s 907ms/step - loss: 0.2517 - acc: 0.9473 - val_loss: 0.0823 - val_acc: 0.9655
Epoch 4/20
1144/1144 [==============================] - 1035s 904ms/step - loss: 0.1482 - acc: 0.9679 - val_loss: 0.0345 - val_acc: 0.9926
Epoch 5/20
1144/1144 [==============================] - 1035s 905ms/step - loss: 0.1082 - acc: 0.9755 - val_loss: 0.0330 - val_acc: 0.9906
Epoch 6/20
1144/1144 [==============================] - 1034s 904ms/step - loss: 0.0731 - acc: 0.9845 - val_loss: 0.0333 - val_acc: 0.9901


In [0]:
xception_reserve = xception

### 4. EfficientNetB0 

In [0]:
from keras_efficientnets import EfficientNetB0

In [0]:
efficientb0 = EfficientNetB0((256, 256, 3), classes=102, include_top=False, weights='imagenet')

In [0]:
efficientb0.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_225 (Conv2D)             (None, 128, 128, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_170 (BatchN (None, 128, 128, 32) 128         conv2d_225[0][0]                 
__________________________________________________________________________________________________
swish_166 (Swish)               (None, 128, 128, 32) 0           batch_normalization_170[0][0]    
__________________________________________________________________________________________________
depthwise_

In [0]:
effb0_avg = GlobalAveragePooling2D()(efficientb0.output)
effb0_fc_1 = Dense(256, activation='relu')(effb0_avg)
effb0_fc_2 = Dense(128, activation='relu')(effb0_fc_1)
effb0_d = Dropout(0.25)(effb0_fc_2)
effb0_fc_3 = Dense(102, activation='softmax')(effb0_d)

In [0]:
efficientb0 = Model(efficientb0.input, outputs  = effb0_fc_3)

In [0]:
efficientb0.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_225 (Conv2D)             (None, 128, 128, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_170 (BatchN (None, 128, 128, 32) 128         conv2d_225[0][0]                 
__________________________________________________________________________________________________
swish_166 (Swish)               (None, 128, 128, 32) 0           batch_normalization_170[0][0]    
__________________________________________________________________________________________________
depthwise_

In [0]:
cp_efficientb0 = keras.callbacks.ModelCheckpoint('efficient_netb0_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)


In [0]:
efficientb0.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
efficientb0_hist = efficientb0.fit_generator(train_gen, steps_per_epoch=18305//16, validation_data=val_gen, validation_steps=235/16, epochs=20, callbacks=[es, cp_efficientb0])

### 5. NASNet

In [0]:
!git clone https://github.com/titu1994/Keras-ResNeXt.git

Cloning into 'Keras-ResNeXt'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), done.


In [0]:
# from google.colab import files
# uploaded = files.upload()

In [0]:
import resnext as res


In [30]:
import keras
from keras_applications.resnext import ResNeXt50

resnext = ResNeXt50(weights='imagenet', input_shape = (256, 256, 3), classes=102, include_top=False, backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

W0821 10:37:44.875379 139885793322880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0821 10:37:44.877959 139885793322880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 10:37:44.888359 139885793322880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 10:37:44.928210 139885793322880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0821 10:37:44.929498 1398857933

92659712/92652672 [==============================] - 1s 0us/step


In [31]:
resnext.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9408        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu

In [32]:
res_avg = GlobalAveragePooling2D()(resnext.output)
res_fc_1 = Dense(256, activation='relu')(res_avg)
res_fc_2 = Dense(128, activation='relu')(res_fc_1)
res_d = Dropout(0.25)(res_fc_2)
res_fc_3 = Dense(102, activation='softmax')(res_d)

W0821 10:38:03.396678 139885793322880 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
resnext = Model(resnext.input, outputs  = res_fc_3)

In [34]:
resnext.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9408        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu

In [35]:
resnext.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W0821 10:38:08.984333 139885793322880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
cp_resnext = keras.callbacks.ModelCheckpoint('resnext.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period = 1)

In [0]:
resnext_hist = resnext.fit_generator(train_gen, steps_per_epoch=17562//8, validation_data=val_gen, validation_steps=978//8, epochs=20, callbacks=[es, cp_resnext])


W0821 10:38:12.671050 139885793322880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
2195/2195 [==============================] - 2776s 1s/step - loss: 1.7563 - acc: 0.6372 - val_loss: 0.0873 - val_acc: 0.9788
Epoch 2/20
2195/2195 [==============================] - 2760s 1s/step - loss: 0.2927 - acc: 0.9380 - val_loss: 0.0230 - val_acc: 0.9906
Epoch 3/20
2195/2195 [==============================] - 2758s 1s/step - loss: 0.1572 - acc: 0.9642 - val_loss: 0.0384 - val_acc: 0.9917
Epoch 4/20
  92/2195 [>.............................] - ETA: 41:51 - loss: 0.1678 - acc: 0.9674

In [0]:
resenext_arcchive = resnext

## Time to make prediction

In [0]:
if not os.path.isdir('data/test/test_data'):
  os.mkdir('data/test/test_data')

In [0]:
# make subdirectory of each class
for each in os.listdir('data/test/'):
  dest = 'data/test/test_data'
  src = os.path.join('data/test', each)
  shutil.move(src, dest)



In [0]:
len(os.listdir('data/test/test_data/'))

2010

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
test_generator = test_datagen.flow_from_directory(
        'data/test',
        shuffle = False,
        class_mode='categorical',
        batch_size=1
)

Found 2009 images belonging to 1 classes.


In [0]:
train_dict = train_gen.class_indices

In [0]:
new_dict = dict([(value, key) for key, value in train_dict.items()])

In [0]:
filenames = test_generator.filenames

In [0]:
nb_samples = len(filenames)

In [0]:
efficientb0.load_weights('efficient_netb0_weights.h5')

In [0]:
predict = resnext.predict_generator(test_generator,steps = nb_samples)

In [0]:
pred = np.argmax(predict, axis=1)

In [0]:
pred.shape

(2009,)

In [0]:
pred[0]

87

In [0]:
actual_pred = [int(new_dict[each]) for each in pred]

In [0]:
actual_pred[0]

86

In [0]:
sample.head(4)

,image_id,category
0,18540,0
1,18541,0
2,18542,0
3,18543,0


In [0]:
sample.category = actual_pred

In [0]:
sample.tail(10)

,image_id,category
1999,20539,40
2000,20540,40
2001,20541,72
2002,20542,51
2003,20543,40
2004,20544,67
2005,20545,51
2006,20546,72
2007,20547,89
2008,20548,63


In [0]:
type(sample)

pandas.core.frame.DataFrame

In [0]:
sample.to_csv('submission.csv', index=False)